This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
import os
import openai
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from typing import Any, Dict
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

os.environ['OPENAI_API_KEY']="YOUR_OPENAI_API_KEY"
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
# Load the documents
file_path = "listings.csv"
loader = CSVLoader(file_path=file_path)
docs = loader.load()

In [3]:
# Split the documents into sentences
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

In [4]:
# Create the vector store
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(split_docs, embeddings)

In [5]:
# Create the LLM
model_name = "gpt-3.5-turbo"
llm = OpenAI(model_name=model_name, temperature=0, max_tokens=2000)

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [6]:
questions = [   
    "How big do you want your house to be?",
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",
    "What is your price range?",
    "How many bathrooms would be ideal?",
    "What is the minimum house size?",
    "How many bedrooms do you need?"
]

answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, open floor plan, and views from backyard.",
    "A backyard for gardening, stainless steel appliances, master suite with a walk-in closet.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and local shops.",
    "Trying to keep it less than $1,000,000.",
    "2 bathrooms is fine, 3 bathrooms would be great.",
    "At least 1800 square feet.",
    "3 or 4 bedrooms"
]

In [7]:
history = ChatMessageHistory()
history.add_user_message(f"""You are AI that will recommend user a new home based on their answers to questions about their home preferences. Ask user {len(questions)} questions""")
for i in range(len(questions)):
    history.add_ai_message(questions[i])
    history.add_user_message(answers[i])

In [8]:
max_rating = 100

summary_memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="recommendation_summary",
    input_key="input",
    buffer=f"The human answered {len(questions)} personal questions. Use them to rate, from 1 to {max_rating}, how much they like a home recommendation.",
    return_messages=True
)

class MementoBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        input_str, output_str = self._get_input_output(inputs, outputs)
        self.chat_memory.add_ai_message(output_str)
        
conversational_memory = MementoBufferMemory(
    chat_memory=history,
    memory_key="questions_and_answers",
    input_key="input"
)

memory = CombinedMemory(memories=[conversational_memory, summary_memory])

In [9]:
user_responses = []
for m in conversational_memory.buffer_as_messages:
    if m.type == "human":
        user_responses.append(m.content)

user_preferences = " ".join(user_responses)

In [10]:
similar_docs = db.similarity_search(user_preferences, k=5)

In [11]:
possible_recommended_homes = "\n############\n".join([f"- {doc.page_content}" for doc in similar_docs])

In [12]:
RECOMMENDER_TEMPLATE = """The following is a friendly conversation between a human and an AI Real Estate Agent. The AI follows human instructions and provides home ratings for a human based on their home preferences. 

Summary of Recommendations:
{recommendation_summary}
Buyer's Preferences Q&A:
{questions_and_answers}
Recommended Homes:
{possible_recommended_homes}
Human: {input}
AI:"""

PROMPT = PromptTemplate.from_template(RECOMMENDER_TEMPLATE).partial(possible_recommended_homes=possible_recommended_homes)

recommender = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=PROMPT)

In [13]:
query = """
Please rank (1-5) and score (0-100) each of the 5 listings based on the buyer's preferences. Provide the output in the following format:

Rank: [Rank]
Score: [Score]
Neighborhood: [Neighborhood]
Price: [Price]
Bedrooms: [Bedrooms]
Bathrooms: [Bathrooms]
Size (sqft): [Size (sqft)]
Description: [Personalized description of the listing based on buyer's preferences. Unique, appealing, tailored to preferences.]
"""

In [14]:
prediction = recommender.predict(input=query)
print(prediction)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI Real Estate Agent. The AI follows human instructions and provides home ratings for a human based on their home preferences. 

Summary of Recommendations:
[SystemMessage(content='The human answered 9 personal questions. Use them to rate, from 1 to 100, how much they like a home recommendation.')]
Buyer's Preferences Q&A:
Human: You are AI that will recommend user a new home based on their answers to questions about their home preferences. Ask user 9 questions
AI: How big do you want your house to be?
Human: A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
AI: What are 3 most important things for you in choosing this property?
Human: A quiet neighborhood, open floor plan, and views from backyard.
AI: Which amenities would you like?
Human: A backyard for gardening, stainless steel appliances, master suite with a walk-in cl